<a href="https://colab.research.google.com/github/petersim1/deep-image-prior/blob/master/conv_vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/Semester\ 3/Bayesian\ ML/Project/flikr8k_sized

/content/drive/MyDrive/Semester 3/Bayesian ML/Project/flikr8k_sized


In [ ]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import MNIST
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
images = []
for f in os.listdir('.'):
  if f != '.DS_Store':
    im = Image.open('./'+f) 
    im = np.asarray(im)
    images.append(im)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, images, mode='train'):
        if mode == 'train':
            self.images = images[:1219]
        elif mode == 'val':
            self.images = images[1219:]
            
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        im = torch.from_numpy(np.moveaxis(images[idx], -1, 0)).float()
        return  im

In [ ]:
train_dataset = ImageDataset(images, 'train')
valid_dataset = ImageDataset(images, 'val')

In [ ]:
train_loader = DataLoader(dataset = train_dataset, batch_size = 1, shuffle = True)
val_loader = DataLoader(dataset = valid_dataset, batch_size = 1, shuffle = False)

In [ ]:
def loss_function(recon_x, x, mu, logvar):
  print('Input max: {}, Recon max: {}'.format(torch.max(x), torch.max(recon_x)))
  BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
  KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

  return BCE + KLD, BCE, KLD

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class UnFlatten(nn.Module):
    def forward(self, input, size=26624):
        return input.view(input.size(0), size, 1, 1)

In [ ]:
class VAE(nn.Module):
    def __init__(self, image_channels=3, h_dim=26624, z_dim=32):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=4, stride=2),
            nn.ReLU(),
            Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim) #26624 = 13x8x256
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(h_dim, 128, kernel_size=(6,9), stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=(6,9), stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=(6,9), stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=(6,9), stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 8, kernel_size=(6,9), stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(8, image_channels, kernel_size=(23,4), stride=2),
            nn.Sigmoid(),
        )
        
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_().to(device)
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size()).to(device)
        z = mu.to(device) + std * esp
        return z
    
    def bottleneck(self, h):
        #this function takes output of encoder and makes two different vectors - one for mu, one for variance
        #result is then passed through reparametrize function
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = self.decoder(z)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar

In [ ]:
device = torch.device('cpu')
print(device)

cpu


In [ ]:
model = VAE().to(device)

learning_rate = 1e-3
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [ ]:
def train(epoch):
    model.train()
    batch_losses = []
    
    for batch_idx, data in enumerate(train_loader):
      data = data.to(device)
      recon_images, mu, logvar = model(data)
      loss, bce, kld = loss_function(recon_images, data, mu, logvar)
      batch_losses.append(loss)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if batch_idx % 50 == 0:
        print('Epoch {}, Batch {}'.format(epoch,batch_idx))

        
    epoch_loss = sum(batch_losses)/len(batch_losses)
    
    return(epoch_loss)

def test(loader):
  model.eval()
  batch_losses = []
  
  with torch.no_grad():
    for batch_idx, (data, labels) in enumerate(loader):
      recon_images, mu, logvar = model(data)
      loss, bce, kld = loss_function(recon_images, data, mu, logvar)
      
      batch_losses.append(loss)
    
    epoch_loss = sum(batch_losses)/len(batch_losses)
    return epoch_loss

In [ ]:
epochs = []
training_losses = []
val_losses = []
for epoch in range(10):
  avg_train_loss = train(epoch)
  print('Train loss for epoch {}: {}'.format(epoch, avg_train_loss))
  val_loss = test(val_loader)
  print('Val loss for epoch {}: {}'.format(epoch, val_loss))
  epochs.append(epoch)
  training_losses.append(avg_train_loss)
  val_losses.append(val_loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Input max: 255.0, Recon max: 0.5143575668334961
Epoch 0, Batch 0
Input max: 255.0, Recon max: nan


RuntimeError: ignored

In [ ]:
%debug

In [ ]:
x = torch.randn((3,3))

In [ ]:
x

In [ ]:
torch.max(x)